# <div style="text-align: right;font-family:Times New Roman">Приложение Б-1</div>
# <div style="text-align: center;font-family:Times New Roman">Предобработка снимков Sentinel-2</div>

# Preprocessing of Sentinel-2 imagery
# Предобработка снимков Sentinel-2
---
In this notebook Sentinel-2 imagery is being preprocessed. Preprocessing includes atmospheric correction using Sen2Cor algorythm, applying superresolution algorythm to 20- and 60-meters bands, cloud masking, reprojecting and mosaic creation.

В этом блокноте выполняется предобработка снимков Sentinel-2. Предобработка включает в себя атмосферную коррекцию с применением алгоритма Sen2Cor, применение алгоритма суперразрешения к каналам с разрешением 20 и 60 м, маскировка облаков, перепроецирование и создание мозаики.

### Importing dependencies
### Импорт необходимых библиотек

In [9]:
import os
import subprocess
import shutil
import random
import re
from typing import List, Tuple
from collections import defaultdict
from pathlib import Path
import numpy as np
from glob import glob
import geopandas as gpd
from osgeo import gdal
import rasterio as rio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import rasterio.merge
import rasterio.fill
from rasterio.io import MemoryFile
from rasterio.enums import Resampling
from rasterio.windows import Window
from rasterio.transform import Affine
import fiona
import shapely
from shapely.geometry import box
import zipfile

### Unzipping all archives with images
### Распаковка всех архивов со снимками

In [79]:
#path to archives / путь к архивам
zippath = 'E:\\Work\\Sentinel\\'
archives = glob(zippath+'*')
#where to unpack / конечный путь распаковки
wpath = 'F:\\Work\\CorrProizv\\LVL1\\'
for archive in archives:
    with zipfile.ZipFile(archive, 'r') as zip_ref:
        zip_ref.extractall(wpath)

### Applying [Sen2Cor](https://step.esa.int/main/snap-supported-plugins/sen2cor/) atmospheric correction algorythm
### Применение алгоритма атмосферной коррекции [Sen2Cor](https://step.esa.int/main/snap-supported-plugins/sen2cor/)

In [3]:
#importing data / импорт данных
wpath = 'F:\\Work\\CorrProizv\\Imagery\\'
paths = glob(wpath+'*')
paths

['F:\\Work\\CorrProizv\\Imagery\\S2A_MSIL2A_20170731T060631_N9999_R134_T44VMN_20210915T184413.SAFE',
 'F:\\Work\\CorrProizv\\Imagery\\S2A_MSIL2A_20170731T060631_N9999_R134_T44VNQ_20210916T052032.SAFE',
 'F:\\Work\\CorrProizv\\Imagery\\S2A_MSIL2A_20170731T060631_N9999_R134_T44VPQ_20210915T081105.SAFE',
 'F:\\Work\\CorrProizv\\Imagery\\S2A_MSIL2A_20180630T072621_N9999_R049_T41VML_20210916T045925.SAFE',
 'F:\\Work\\CorrProizv\\Imagery\\S2A_MSIL2A_20180723T055641_N9999_R091_T44VNP_20210915T192826.SAFE',
 'F:\\Work\\CorrProizv\\Imagery\\S2A_MSIL2A_20180826T071621_N9999_R006_T41VMK_20210915T093843.SAFE',
 'F:\\Work\\CorrProizv\\Imagery\\S2A_MSIL2A_20190701T060641_N9999_R134_T44VMN_20210915T070556.SAFE',
 'F:\\Work\\CorrProizv\\Imagery\\S2A_MSIL2A_20190701T060641_N9999_R134_T44VMR_20210915T195148.SAFE',
 'F:\\Work\\CorrProizv\\Imagery\\S2A_MSIL2A_20190715T054641_N9999_R048_T44VPN_20210915T042924.SAFE',
 'F:\\Work\\CorrProizv\\Imagery\\S2A_MSIL2A_20190721T060641_N9999_R134_T43VFJ_20210915T0121

In [ ]:
#creating env for running Sen2Cor / создание среды для запуска Sen2Cor
my_env = os.environ.copy()
my_env['PATH'] = r'C:\ProgramData\Sen2Cor-02.09.00-win64\bin;' + r'C:\ProgramData\Sen2Cor-02.09.00-win64;' + my_env['PATH']
my_env['SEN2COR_HOME'] = r'C:\ProgramData\Sen2Cor-02.09.00-win64'
my_env['SEN2COR_BIN'] = r'C:\ProgramData\Sen2Cor-02.09.00-win64\lib\python2.7\site-packages\sen2cor'
my_env['LC_NUMERIC'] = 'C'
my_env['GDAL_DATA'] = r'C:\ProgramData\Sen2Cor-02.09.00-win64\share\gdal'
my_env['GDAL_DRIVER_PATH']= 'disable'

In [ ]:
# running Sen2Cor for every image / применение Sen2Cor к каждому снимку
for path in paths:
    cmd = r'L2A_Process.bat "'+ path + '" '
    try:
        result = subprocess.check_output(cmd, env=my_env, shell=True)
        shutil.rmtree(path)
        print(path)
    except subprocess.CalledProcessError as e:
        print(e)

### Applying superresolution algorythm
### Применение алгоритма суперразрешения

In [ ]:
#importing python library for superresolution algorythm / импорт библиотеки, реализующей алгоритм суперразрешения
#https://github.com/simonreise/s2-superres

import sys
sys.path.insert(1, r'C:\Users\mosko\Downloads\s2-superres\src\s2_superres')
import s2_superres

In [ ]:
out_dir = r'F:\Work\CorrProizv\superres\\'

#applying superresolution to every image / применяем алгоритм суперразрешения к каждому снимку
for path in paths:
    pref = Path(path).stem
    if (Path(out_dir + pref + '_superresolution.tif').is_file()):
        print('Pass: ' + path)
    else:
        s2_superres.Superresolution(input_dir = path, output_dir = out_dir, copy_original_bands = True).start()
        print('DONE: ' + path)
        print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')

### Cloud masking, errors fixing and reprojection
### Маскировка облаков, исправление технических ошибок и перепроецирование

In [5]:
for path in paths:
    #loading bands / загрузка всех каналов
    if os.path.isfile('F:\\Work\\CorrProizv\\projected\\' + path.split('\\')[4][:-5] + '\\B1.tif'):
        print('Skip', path)
    else:
        img = 'F:\\Work\\CorrProizv\\superres\\' + path.split('\\')[4][:-5] + '_superresolution.tif'
        with rio.open(img) as bands:
            img = bands.read()
            meta = bands.profile
            bounds = bands.bounds
            descs = bands.descriptions
            transform = bands.transform
        #masking zeros / маскировка нулей
        img = np.where(img == 0, 1, img)
        #masking 65555 pixels / маскировка значений 65555
        mask = np.where(img > 15000, 0, 1)
        for i in range(img.shape[0]):
            img[i] = rio.fill.fillnodata(img[i], mask[i], max_search_distance = 500)
        #masking clouds by Sentinel L1 mask / маскировка облаков при помощи всроенной маски Sentinel L1
        try:
            shape = glob(path + '\\GRANULE\\**\\QI_DATA\\MSK_CLOUDS_B00.gml')[0]
            with fiona.open(shape, "r") as shapefile:
                cloudmask = [feature["geometry"] for feature in shapefile]
            with MemoryFile() as memfile:
                with memfile.open(
                    driver='GTiff',
                    height=img.shape[1],
                    width=img.shape[2],
                    count=img.shape[0],
                    dtype=img.dtype,
                    compress = 'lzw',
                    crs=meta['crs'],
                    transform=transform
                ) as temp:
                    temp.write(img)
                    img, transform = rio.mask.mask(temp, cloudmask, invert = True, filled = True)
        except:
            pass
        #masking clouds by algorythm based on Scene Classification of L2 product and bands 1 and 9
        #маскировка облаков при помощи алгоритма, основанного на классификации снимка уровня 2 и каналов 1 и 9
        l1path = glob(path + r'\GRANULE\*\IMG_DATA\R20m\*SCL_20m.jp2')
        with rio.open(l1path[0]) as classification:
            cls = classification.read()
            clsmeta = classification.profile
        mask = np.empty_like(img[0])
        reproject(
            cls, mask,
            src_transform = clsmeta['transform'],
            dst_transform = meta['transform'],
            src_crs = clsmeta['crs'],
            dst_crs = meta['crs'],
            resampling = Resampling.nearest)
        for i in range(len(descs)):
            if 'B1 ' in descs[i]:
                band1 = i
        for i in range(len(descs)):
            if 'B9 ' in descs[i]:
                band9 = i
        clmask = np.where(((img[band1] >= 3200) & (img[band9] >= 6000) & (mask >= 7)), 1, 0)
        #delete 1px gaps in mask / удаление пробелов в 1 пиксел
        clmask = np.where((rio.fill.fillnodata(np.where(clmask == 1, 0, 1), np.where(clmask == 1, 0, 1), max_search_distance = 2)) == 1, 0, 1)
        #emerge mask / увеличение маски
        clmask = rio.fill.fillnodata(clmask, clmask, max_search_distance = 22)
        img = np.where(clmask > 0, 0, img)
        #masking nodata / маскировка
        img = np.where(mask == 0, 0, img)
        #reprojecting / перепроецирование
        if meta['crs'] != 'EPSG:32642':
            transform, width, height = calculate_default_transform(
                meta['crs'], 'EPSG:32642', meta['width'], meta['height'], *bounds)
            proj = np.zeros((img.shape[0], width, height), img.dtype)
            img, transform = reproject(
                source=img,
                destination=proj,
                src_transform=meta['transform'],
                src_crs=meta['crs'],
                dst_transform=transform,
                dst_crs='EPSG:32642',
                resampling=Resampling.nearest)
            img = proj
        #save / сохранение
        Path('F:\\Work\\CorrProizv\\projected\\' + path.split('\\')[4][:-5]).mkdir(parents=True, exist_ok=True)
        for i in range(img.shape[0]):
            pathres = 'F:\\Work\\CorrProizv\\projected\\' + path.split('\\')[4][:-5] + '\\' + descs[i].split(' ')[1] + '.tif'
            with rio.open(
                pathres,
                'w',
                driver='GTiff',
                height=img.shape[1],
                width=img.shape[2],
                count=1,
                dtype=img.dtype,
                compress = 'deflate',
                PREDICTOR = 2,
                ZLEVEL=9,
                crs='EPSG:32642',
                transform=transform,
                nodata = 0
            ) as outfile:
                outfile.write(img[i],1)
        print(path)

F:\Work\CorrProizv\Imagery\S2A_MSIL2A_20170731T060631_N9999_R134_T44VMN_20210915T184413.SAFE
F:\Work\CorrProizv\Imagery\S2A_MSIL2A_20170731T060631_N9999_R134_T44VNQ_20210916T052032.SAFE
F:\Work\CorrProizv\Imagery\S2A_MSIL2A_20170731T060631_N9999_R134_T44VPQ_20210915T081105.SAFE
F:\Work\CorrProizv\Imagery\S2A_MSIL2A_20180630T072621_N9999_R049_T41VML_20210916T045925.SAFE
F:\Work\CorrProizv\Imagery\S2A_MSIL2A_20180723T055641_N9999_R091_T44VNP_20210915T192826.SAFE
F:\Work\CorrProizv\Imagery\S2A_MSIL2A_20180826T071621_N9999_R006_T41VMK_20210915T093843.SAFE
F:\Work\CorrProizv\Imagery\S2A_MSIL2A_20190701T060641_N9999_R134_T44VMN_20210915T070556.SAFE
F:\Work\CorrProizv\Imagery\S2A_MSIL2A_20190701T060641_N9999_R134_T44VMR_20210915T195148.SAFE
F:\Work\CorrProizv\Imagery\S2A_MSIL2A_20190715T054641_N9999_R048_T44VPN_20210915T042924.SAFE
F:\Work\CorrProizv\Imagery\S2A_MSIL2A_20190721T060641_N9999_R134_T43VFJ_20210915T012154.SAFE
F:\Work\CorrProizv\Imagery\S2A_MSIL2A_20190721T060641_N9999_R134_T44VL

F:\Work\CorrProizv\Imagery\S2A_MSIL2A_20200717T064631_N9999_R020_T42VVL_20210915T125706.SAFE
F:\Work\CorrProizv\Imagery\S2A_MSIL2A_20200717T064631_N9999_R020_T42VVM_20210913T160650.SAFE
F:\Work\CorrProizv\Imagery\S2A_MSIL2A_20200717T064631_N9999_R020_T42VXR_20210913T145941.SAFE
F:\Work\CorrProizv\Imagery\S2A_MSIL2A_20200717T064631_N9999_R020_T43VCK_20210913T152148.SAFE
F:\Work\CorrProizv\Imagery\S2A_MSIL2A_20200717T064631_N9999_R020_T43VCL_20210913T154448.SAFE
F:\Work\CorrProizv\Imagery\S2A_MSIL2A_20200717T064631_N9999_R020_T43VDL_20210913T162838.SAFE
F:\Work\CorrProizv\Imagery\S2A_MSIL2A_20200719T072621_N9999_R049_T41VMJ_20210913T143930.SAFE
F:\Work\CorrProizv\Imagery\S2A_MSIL2A_20200802T070631_N9999_R106_T41VPL_20210913T141855.SAFE
F:\Work\CorrProizv\Imagery\S2A_MSIL2A_20200803T063631_N9999_R120_T43VCF_20210913T125627.SAFE
F:\Work\CorrProizv\Imagery\S2A_MSIL2A_20200804T060641_N9999_R134_T44VMP_20210913T123421.SAFE
F:\Work\CorrProizv\Imagery\S2A_MSIL2A_20200805T053651_N9999_R005_T45VV

### Merging all Sentinel-2 images into one mosaic for whole area
### Объединение всех снимков Sentinel-2 в одну мозаику на всю территорию

In [6]:
#getting all images / список всех снимков
spaths = glob('F:\\Work\\CorrProizv\\Projected\\'+'*')
spaths

['F:\\Work\\CorrProizv\\Projected\\S2A_MSIL2A_20170731T060631_N9999_R134_T44VMN_20210915T184413',
 'F:\\Work\\CorrProizv\\Projected\\S2A_MSIL2A_20170731T060631_N9999_R134_T44VNQ_20210916T052032',
 'F:\\Work\\CorrProizv\\Projected\\S2A_MSIL2A_20170731T060631_N9999_R134_T44VPQ_20210915T081105',
 'F:\\Work\\CorrProizv\\Projected\\S2A_MSIL2A_20180630T072621_N9999_R049_T41VML_20210916T045925',
 'F:\\Work\\CorrProizv\\Projected\\S2A_MSIL2A_20180723T055641_N9999_R091_T44VNP_20210915T192826',
 'F:\\Work\\CorrProizv\\Projected\\S2A_MSIL2A_20180826T071621_N9999_R006_T41VMK_20210915T093843',
 'F:\\Work\\CorrProizv\\Projected\\S2A_MSIL2A_20190701T060641_N9999_R134_T44VMN_20210915T070556',
 'F:\\Work\\CorrProizv\\Projected\\S2A_MSIL2A_20190701T060641_N9999_R134_T44VMR_20210915T195148',
 'F:\\Work\\CorrProizv\\Projected\\S2A_MSIL2A_20190715T054641_N9999_R048_T44VPN_20210915T042924',
 'F:\\Work\\CorrProizv\\Projected\\S2A_MSIL2A_20190721T060641_N9999_R134_T43VFJ_20210915T012154',
 'F:\\Work\\CorrProi

###### Sorting images by cloud cover - less cloudy on top
###### Сортировка снимков по облачности - наименее облачные сверху

In [7]:
#getting number of nodata pixels for each image / количество значений "нет данных" для каждого снимка
zeros = []
for path in spaths:
    path = path + '\\B1.tif'
    with rio.open(path) as bnd:
        img = bnd.read(1)
    try:
        zeros.append(np.count_nonzero(img==0))
    except:
        zeros.append(0)

In [8]:
#sorting images by number of nodata pixels / сортировка снимков по количеству пикселов "нет даннных"
zerosdict = dict(zip(spaths, zeros))
sortedzeros = dict(sorted(zerosdict.items(), key=lambda item: item[1], reverse = True))
order = list(sortedzeros.keys())
order

['F:\\Work\\CorrProizv\\Projected\\S2A_MSIL2A_20190721T060641_N9999_R134_T44VLQ_20210915T073007',
 'F:\\Work\\CorrProizv\\Projected\\S2A_MSIL2A_20190721T060641_N9999_R134_T45VUK_20210915T024042',
 'F:\\Work\\CorrProizv\\Projected\\S2A_MSIL2A_20190725T072621_N9999_R049_T40WFT_20210915T020246',
 'F:\\Work\\CorrProizv\\Projected\\S2A_MSIL2A_20200710T065631_N9999_R063_T41VNG_20210914T032628',
 'F:\\Work\\CorrProizv\\Projected\\S2B_MSIL2A_20200608T070629_N9999_R106_T41VMH_20210914T090442',
 'F:\\Work\\CorrProizv\\Projected\\S2B_MSIL2A_20200617T055639_N9999_R091_T44VMN_20210914T121732',
 'F:\\Work\\CorrProizv\\Projected\\S2A_MSIL2A_20190820T074611_N9999_R135_T41WMN_20210914T132020',
 'F:\\Work\\CorrProizv\\Projected\\S2B_MSIL2A_20190819T072619_N9999_R049_T40WFS_20210914T181115',
 'F:\\Work\\CorrProizv\\Projected\\S2A_MSIL2A_20190813T061631_N9999_R034_T43VDG_20210914T212730',
 'F:\\Work\\CorrProizv\\Projected\\S2A_MSIL2A_20170731T060631_N9999_R134_T44VPQ_20210915T081105',
 'F:\\Work\\CorrProi

In [ ]:
#merging images into one mosaic / объединение снимков
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12']
for band in bands:
    #reading n-th band from every image / чтение n-ного канала каждого из снимков
    ordfiles = []
    for img in order:
        img = img + '\\' + band + '.tif'
        ordfiles.append(img)
    files = []
    for path in ordfiles:
        pathfile = rio.open(path)
        files.append(pathfile)
    #merging / объединение
    final, final_trans = rio.merge.merge(files,method = 'last')
    print('merge done')
    #filling the gaps (if exist) / заполнение пустот (если они есть)
    final = rio.fill.fillnodata(final, final, max_search_distance=250)
    print('fill done')
    #clipping the mosaic to the extent of KhMAO / обрезка мозаики по границе ХМАО
    files = None
    memfile = None
    temp = None
    with fiona.open("E:\\Work\\CorrProizv\\clipper.gpkg", "r") as shapefile:
        shape = [feature["geometry"] for feature in shapefile]
    with MemoryFile() as memfile:
        with memfile.open(
            driver='GTiff',
            height=final.shape[1],
            width=final.shape[2],
            count=final.shape[0],
            dtype=final.dtype,
            compress = 'lzw',
            crs='EPSG:32642',
            transform=final_trans,
            BIGTIFF='YES',
            nodata = 0
        ) as temp:
            temp.write(final)
            final, final_trans = rio.mask.mask(temp, shape,crop=True, filled=True)
    print('clipping done')
    #saving / сохранение
    with rio.open(
        'F:\\Work\\CorrProizv\\final_noadj\\'+ band + '.tif',
        'w',
        driver='GTiff',
        height=final.shape[1],
        width=final.shape[2],
        count=1,
        dtype=final.dtype,
        compress = 'lzw',
        crs='EPSG:32642',
        transform=final_trans,
        BIGTIFF='YES',
        nodata = 0
    ) as outfile:
        outfile.write(final)
    print(band)

merge done


#### Sentinel-2 processing tools from this notebook are now avaliable in a Python library named [Remote Sensing Processor](https://github.com/simonreise/remote-sensing-processor)
#### Инструменты предобработки Sentinel-2, использованные в этом блокноте доступны в библиотеке Python [Remote Sensing Processor](https://github.com/simonreise/remote-sensing-processor)